# Setup and load data:

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#%cd /dance

import os
import sys
import warnings

print(os.getcwd())

sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings('ignore')

/content


In [4]:
%cd /content/drive/MyDrive/colab-github/move/dance

/content/drive/.shortcut-targets-by-id/1-DJonK3jse0NWdbyWWAIrK_srm2hV5ug/move/dance


In [5]:
!pip install torchinfo

In [6]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
from tqdm import tqdm
import torch
import tensorflow as tf
from functions.draft_torch_lstm import *


In [7]:
torch.cuda.is_available()


True

In [8]:
print(os.getcwd())

ds_all, ds_all_centered, datasets, datasets_centered, ds_counts = load_data()

/content/drive/.shortcut-targets-by-id/1-DJonK3jse0NWdbyWWAIrK_srm2hV5ug/move/dance
loading: betternot_and_retrograde
	 Shape: (9925, 53, 3)
	 Min: [-1.96127391 -4.00212193 -0.15710293]
	 Max: [2.1591475  3.52660036 2.09531498]
loading: knownbetter
	 Shape: (6649, 53, 3)
	 Min: [-2.42928696 -3.50348401 -0.48134506]
	 Max: [2.13536358 3.36868167 2.0231936 ]
Full data shape: (16574, 53, 3)


In [9]:
torch.cuda.is_available()

True

In [10]:
ds_all.shape
my_data = ds_all.reshape((ds_all.shape[0], -1))
my_data.shape

(16574, 159)

In [11]:
#Make seq_data that will have shape 
# [number of seq, 128, input_features]
seq_len=128
seq_data = np.zeros((my_data.shape[0]-seq_len, seq_len, my_data.shape[1]))
for i in range((ds_all.shape[0]-seq_len)):
    seq_data[i] = my_data[i:i+seq_len]


In [12]:
seq_data.shape

(16446, 128, 159)

# Build the autoencoder for poses

In [13]:
#Initialize encoder and decoder
my_encoder = LstmEncoder(input_features=3*53, h_features_loop=32, latent_dim=32)
my_decoder = LstmDecoder(n_layers=2, output_features=3*53, h_features_loop=32, latent_dim=32, seq_len=128)
my_lstmvae = LstmVAE()

# Running the Autoencoder

Set the batch size and learning rate.

In [15]:
batchsize = 8
learning_rate= 3e-5

In [16]:
import torch
data_torch = torch.utils.data.DataLoader(seq_data, 
    batch_size=batchsize, num_workers=2)

x_input = torch.tensor(data_torch.dataset, dtype=torch.float32)


In [17]:
z_torch, mu, logvar = my_encoder(x_input[:8])
#add print statements to see where it crashes or infinitely loops
#setup_gpus
#do we need to update driver?

starting the forward of encoder. the first step is calling layer lstm1
done layer lstm1. It returned h1 of shape torch.Size([8, 128, 32]) and h1_T of shapetorch.Size([1, 8, 32])
Now starting the loop of 2-1 lstm layers
this is loop iteration 0. Calling layer lstm2
done layer lstm 2. lstm2 returns h2 of shape torch.Size([8, 128, 32]) and h2_T of shape torch.Size([1, 8, 32])
Now computing the encoder output.
calling mean_block
z_mean has shape torch.Size([1, 8, 32])
z_logvar has shape torch.Size([1, 8, 32])
reparametrize function called
made std
print made eps
z_sample has shape torch.Size([1, 8, 32])
encoder is done


In [18]:
print(z_torch.shape)

torch.Size([1, 8, 32])


In [19]:
x_recon = my_decoder(z_torch)
print(x_recon.shape)


torch.Size([128, 8, 159])


In [20]:
x_mean, z_sample, z_mean, z_log_var = my_lstmvae(x_input[:8])

starting the forward of encoder. the first step is calling layer lstm1
done layer lstm1. It returned h1 of shape torch.Size([8, 128, 32]) and h1_T of shapetorch.Size([1, 8, 32])
Now starting the loop of 2-1 lstm layers
this is loop iteration 0. Calling layer lstm2
done layer lstm 2. lstm2 returns h2 of shape torch.Size([8, 128, 32]) and h2_T of shape torch.Size([1, 8, 32])
Now computing the encoder output.
calling mean_block
z_mean has shape torch.Size([1, 8, 32])
z_logvar has shape torch.Size([1, 8, 32])
reparametrize function called
made std
print made eps
z_sample has shape torch.Size([1, 8, 32])
encoder is done


In [21]:
i = 0

print(data_torch.dataset.shape)
for x in data_torch:
    print(x.shape)

(16446, 128, 159)
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 159])
torch.Size([8, 128, 

In [ ]:
model = my_lstmvae
from torch.autograd import Variable

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
loss_array=[]


for epoch in range(10):
    model.train()
    total_loss = 0
    for x in data_torch:
        x = Variable(x)

        x_recon, z, z_mu, z_logvar = my_lstmvae(x.float())
        x_recon_batch_first=x_recon.reshape((x_recon.shape[1], x_recon.shape[0],x_recon.shape[2]))
        print(x_recon_batch_first.shape)
        loss = torch.mean(model.elbo(x, x_recon_batch_first, z, (z_mu, z_logvar)))
        loss.backward()
        loss_array.append(loss.item())

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss


loss_real_array = np.asarray(loss_array)
x_axis = np.arange(0, len(loss_real_array))
plt.figure(figsize=(12,8))
plt.plot(x_axis, loss_real_array)
plt.ylabel('Loss')
plt.xlabel('Batch')
plt.title('Batch size = {}, Learning rate = {}'.format(batchsize, learning_rate))
plt.savefig('Training_Loss_vs_Batch/batch_{}_lr_{}.png'.format(batchsize, learning_rate))      

Streaming output truncated to the last 5000 lines.
calling mean_block
z_mean has shape torch.Size([1, 8, 32])
z_logvar has shape torch.Size([1, 8, 32])
reparametrize function called
made std
print made eps
z_sample has shape torch.Size([1, 8, 32])
encoder is done
torch.Size([8, 128, 159])
x_in has shape torch.Size([8, 128, 159]) and x_out has shape torch.Size([8, 128, 159])
starting the forward of encoder. the first step is calling layer lstm1
done layer lstm1. It returned h1 of shape torch.Size([8, 128, 32]) and h1_T of shapetorch.Size([1, 8, 32])
Now starting the loop of 2-1 lstm layers
this is loop iteration 0. Calling layer lstm2
done layer lstm 2. lstm2 returns h2 of shape torch.Size([8, 128, 32]) and h2_T of shape torch.Size([1, 8, 32])
Now computing the encoder output.
calling mean_block
z_mean has shape torch.Size([1, 8, 32])
z_logvar has shape torch.Size([1, 8, 32])
reparametrize function called
made std
print made eps
z_sample has shape torch.Size([1, 8, 32])
encoder is done


In [ ]:
import matplotlib.pyplot as plt
loss_real_array = np.asarray(loss_array)
x_axis = np.arange(0, len(loss_real_array))
plt.figure(figsize=(12,8))
plt.plot(x_axis, loss_real_array)
plt.ylabel('Loss')
plt.xlabel('Batch')
plt.title('Batch size = {}, Learning rate = {}'.format(batchsize, learning_rate))
plt.savefig('Training_Loss_vs_Batch/batch_{}_lr_{}.png'.format(batchsize, learning_rate))
plt.show()

<Figure size 1200x800 with 1 Axes>

## Mariel's Autoencoder

In [ ]:
import numpy as np
from keras import backend as K
from keras.layers import Dense, Dropout, Flatten, Input, Lambda, LeakyReLU, Reshape
from keras.models import Model
from tensorflow.keras.optimizers import Adam


class Autoencoder:
    def __init__(
        self,
        n_verts=0,
        n_dims=3,
        latent_dim=2,
        n_layers=2,
        n_units=128,
        relu=False,
        add_random_offsets=False,
        dropout=False,
    ):
        if not n_verts:
            raise Exception("Please provide the number of vertices `n_verts`")
        self.n_verts = n_verts  # input vert count
        self.n_dims = n_dims  # input dimensions
        self.relu = relu
        # whether to add relu layers in encoder/decoder
        self.dropout = dropout  # whether to add dropout layers in encoder/decoder
        self.latent_dim = latent_dim
        self.n_layers = n_layers
        self.n_units = n_units
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        # attach the encoder and decoder
        i = Input((self.n_verts, self.n_dims))
        if add_random_offsets:
            random_offsets = (
                K.cast(K.learning_phase(), "float")
                * K.random_uniform((K.shape(i)[0], 1, 3))
                * K.constant([[[1, 1, 0]]])
            )
            offset_layer = Lambda(lambda x: x + random_offsets)
            offset_layer.uses_learning_phase = True
            i_offset = offset_layer(i)
        else:
            i_offset = i
        z = self.encoder(i_offset)  # push observations into latent space
        o = self.decoder(z)  # project from latent space to feature space
        if add_random_offsets:
            o = Lambda(lambda x: x - random_offsets)(o)
        self.model = Model(inputs=[i], outputs=[o])
        self.model.compile(loss="mse", optimizer=Adam(lr=1e-4))

    def build_encoder(self):
        i = Input((self.n_verts, self.n_dims))
        h = i
        h = Flatten()(h)
        for _ in range(self.n_layers):
            h = Dense(self.n_units)(h)
            if self.relu:
                h = LeakyReLU(alpha=0.2)(h)
            if self.dropout:
                h = Dropout(0.4)(h)
        o = Dense(self.latent_dim)(h)
        return Model(inputs=[i], outputs=[o])

    def build_decoder(self):
        i = Input((self.latent_dim,))
        h = i
        for _ in range(self.n_layers):
            h = Dense(self.n_units)(h)
            if self.relu:
                h = LeakyReLU(alpha=0.2)(h)
            if self.dropout:
                h = Dropout(0.4)(h)
        h = Dense(self.n_verts * self.n_dims)(h)
        o = Reshape((self.n_verts, self.n_dims))(h)  # predict 1 frame
        return Model(inputs=[i], outputs=[o])

    def train(self, X, n_epochs=10000):
        for idx in range(n_epochs):
            i = np.random.randint(0, X.shape[1] - 1)  # sample idx
            frame = np.expand_dims(
                X[:, i : i + 1, :].squeeze(), axis=0
            )  # shape = 1 sample, v verts, d dims
            loss = self.model.train_on_batch(frame, frame)
            if idx == 0:
                print(frame.shape)
            if idx % 1000 == 0:
                print(" * training idx", idx, "loss", loss)

    def get_predictions(self, X, n_frames=50, start_frame=0):
        """Return the model's predictions of observations from X in shape of X"""
        predictions = []
        for i in range(start_frame, start_frame + n_frames, 1):
            x = np.expand_dims(X[:, i : i + 1, :].squeeze(), axis=0)
            predictions.append(self.model.predict(x))
        return np.swapaxes(np.vstack(predictions), 0, 1)
